# Convert multiple PBT logs → machine-readable CSV + Excel workbook

This notebook handles **multiple PBT log files** and **adds a `seed` column** inferred from the filename
(e.g., `bs+wd_seed_38042.log` → seed `38042`).

For each seed/log, we parse:
- per-epoch/per-member metrics
- PBT update events (hyperparameter changes + copy events)
- optional post-update hyperparameter lines
- population-level epoch summaries

Then we reconstruct **per-epoch PBT hyperparameters** (since they change over time), concatenate all seeds,
and export:
- `results/pbt_bs_wd_parsed.csv`
- `results/pbt_bs_wd_parsed.xlsx` (with leaderboards, plus per-seed top-200 sheets)


In [1]:
from pathlib import Path

# =============================================================================
# CONFIGURATION
# =============================================================================

# Global variable: relative directory where outputs will be written.
CSV_REL_DIR = "../Structured Outputs/PBT/"

# Provide one or more PBT log files here. Seed is inferred from the filename.
# Example names: bs+wd_seed_38042.log, bs+wd_seed_217401.log
COMMON_PATH = Path("../Raw Outputs/PBT/Full Logs/")
INPUT_LOG_PATHS = [
    COMMON_PATH / "bs_seed_38042.log",
    COMMON_PATH / "bs_seed_217401.log",
]

# Output names (written inside CSV_REL_DIR)
OUTPUT_CSV_NAME = "pbt_bs_parsed.csv"
OUTPUT_XLSX_NAME = "pbt_bs_parsed.xlsx"

# Derived paths
OUTPUT_DIR = Path(CSV_REL_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_CSV_PATH = OUTPUT_DIR / OUTPUT_CSV_NAME
OUTPUT_XLSX_PATH = OUTPUT_DIR / OUTPUT_XLSX_NAME

print("Will read:", INPUT_LOG_PATHS)
print("Will write CSV:", OUTPUT_CSV_PATH)
print("Will write XLSX:", OUTPUT_XLSX_PATH)


Will read: [PosixPath('../Raw Outputs/PBT/Full Logs/bs_seed_38042.log'), PosixPath('../Raw Outputs/PBT/Full Logs/bs_seed_217401.log')]
Will write CSV: ../Structured Outputs/PBT/pbt_bs_parsed.csv
Will write XLSX: ../Structured Outputs/PBT/pbt_bs_parsed.xlsx


## Parser implementation

We implement a function `parse_pbt_log(path)` which:
1. Infers `seed` from the filename (`seed_XXXXX`).
2. Parses:
   - `(epoch, member)` metrics
   - PBT hyperparameter change events (`old -> new`) per member and per update epoch
   - copy events
   - population summaries
3. Reconstructs per-epoch PBT hyperparameters (`pbt_lr`, `pbt_weight_decay`, `pbt_drop_path`, `pbt_batch_size`)
   because PBT mutates hyperparameters over time.


In [2]:
# -------------------------------
# Parsing utilities (UPDATED)
# -------------------------------
from pathlib import Path
import re
import numpy as np
import pandas as pd

# Epoch + member blocks
EPOCH_HDR = re.compile(r"\bEpoch\s+(?P<epoch>\d+)\s*/\s*(?P<epoch_total>\d+)\b", re.IGNORECASE)
TRAIN_MEMBER = re.compile(
    r"---\s*Training\s+Member\s+(?P<member>\d+)\s*\(Batch\s+size:\s*(?P<bs>\d+)\)\s*---",
    re.IGNORECASE,
)

# LR + batch size are taken ONLY from per-member training blocks:
LR_CHANGED = re.compile(
    r"LR\s+changed\s+during\s+epoch:\s*(?P<start>[-+0-9.eE]+)\s*->\s*(?P<end>[-+0-9.eE]+)",
    re.IGNORECASE,
)

# Per-member metrics
BUILT = re.compile(r"built\s+data\s+in\s+(?P<t>[-+0-9.eE]+)\s+seconds", re.IGNORECASE)
TRAIN_TIME = re.compile(r"total\s+runtime\s+to\s+train\s+this\s+model\s+was\s+(?P<t>[-+0-9.eE]+)\s+seconds", re.IGNORECASE)
EVAL_TIME = re.compile(r"evaluation\s+in\s+(?P<t>[-+0-9.eE]+)\s+seconds", re.IGNORECASE)
LOSS = re.compile(r"Loss:\s*(?P<loss>[-+0-9.eE]+)", re.IGNORECASE)
TRAIN_ACC = re.compile(r"Train\s+Accuracy:\s*(?P<acc>[-+0-9.]+)\s*%", re.IGNORECASE)
TEST_ACC = re.compile(r"Test\s+Accuracy:\s*(?P<acc>[-+0-9.]+)\s*%", re.IGNORECASE)

# Population update blocks (non-LR/non-batch hyperparams come from here)
POP_UPDATE_HDR = re.compile(r"---\s*Population\s+Update\s*\(Epoch\s+(?P<epoch>\d+)\)\s*---", re.IGNORECASE)
CHANGE_LINE = re.compile(
    r"Member\s+(?P<member>\d+):\s*(?P<param>[A-Za-z0-9_]+)\s+changed\s+from\s+(?P<old>[-+0-9.eE]+)\s+to\s+(?P<new>[-+0-9.eE]+)",
    re.IGNORECASE,
)
COPIED_LINE = re.compile(r"Member\s+(?P<member>\d+)\s+copied\s+from\s+(?P<src>\d+)", re.IGNORECASE)
POST_LINE = re.compile(
    r"LR=(?P<lr>[-+0-9.eE]+),\s*WD=(?P<wd>[-+0-9.eE]+),\s*DropPath=(?P<dp>[-+0-9.eE]+),\s*Warmup=(?P<warm>\d+)\s*epochs,\s*Batch=(?P<bs>\d+)",
    re.IGNORECASE,
)

# Epoch summary blocks
SUMMARY_HDR = re.compile(r"Epoch\s+(?P<epoch>\d+)\s+Summary:", re.IGNORECASE)
SUMMARY_TIME = re.compile(r"Time:\s*(?P<time>[-+0-9.]+)s\s*\(Avg\s+member:\s*(?P<avg>[-+0-9.]+)s\)", re.IGNORECASE)
POP_MEAN_ACC = re.compile(r"Population\s+Mean\s+Accuracy:\s*(?P<acc>[-+0-9.]+)\s*%", re.IGNORECASE)
BEST_MEMBER_ACC = re.compile(r"Best\s+Member\s+Accuracy:\s*(?P<acc>[-+0-9.]+)\s*%", re.IGNORECASE)
MEAN_BS = re.compile(r"Mean\s+Batch\s+Size:\s*(?P<bs>\d+)", re.IGNORECASE)
MEAN_LR = re.compile(r"Mean\s+Learning\s+Rate:\s*(?P<lr>[-+0-9.eE]+)", re.IGNORECASE)
MEAN_WD = re.compile(r"Mean\s+Weight\s+Decay:\s*(?P<wd>[-+0-9.eE]+)", re.IGNORECASE)

SEED_RE = re.compile(r"seed[_=](\d+)", re.IGNORECASE)

def _norm_param_name(p: str) -> str:
    p = p.lower().strip()
    if p in ("weightdecay", "weight_decay", "wd"):
        return "weight_decay"
    if p in ("drop_path", "droppath", "drop_path_rate", "dpr"):
        return "drop_path"
    if p in ("warmup", "warmup_epochs", "warmup_epoch"):
        return "warmup_epochs"
    if p in ("lr", "learning_rate", "learningrate"):
        return "lr"
    if p in ("batch", "batch_size", "bs"):
        return "batch_size"
    return p

def parse_seed_from_name(p: Path) -> int:
    m = SEED_RE.search(p.name)
    if not m:
        raise ValueError(f"Could not infer seed from filename: {p.name}")
    return int(m.group(1))

def parse_pbt_text(text: str, seed: int) -> dict:
    """
    Parse a PBT log into:
      - df_main: per-epoch, per-member metrics + hyperparams
      - df_changes: hyperparameter-change events (from Population Update blocks)
      - df_copies: copy events (from Population Update blocks)
      - df_post: post-exploit hyperparam summary lines
      - df_summary: epoch-level population summary
    Policy:
      - LR + batch size ONLY from per-member training blocks.
      - All other hyperparameters ONLY from Population Update blocks.
      - Non-(LR, batch) hyperparams are forward-filled, then backfilled per member to
        remove leading NaNs (per the requested backfill behavior).
      - When a member is copied from another, we also anchor the *source* member's
        non-(LR, batch) hyperparams using the destination's post line for backfill.
    """
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]

    # state tracks non-(LR, batch) hyperparams at the start of each epoch for each member
    state = {}  # member -> dict of {"weight_decay":..., "drop_path":..., "warmup_epochs":...}

    def ensure(m: int):
        if m not in state:
            state[m] = {}

    rows = []
    changes = []
    copies = []
    post_rows = []
    summary_rows = []
    anchors = []  # for backfilling source members: {"member":src, "epoch":u, ...}

    current_epoch = None
    epoch_total = None
    current_member = None
    last_row_idx = None

    in_pop_update = False
    update_epoch = None
    pending_updates = {}  # dest_member -> {param: new_value} (non-(LR,batch) only)
    copied_from = {}      # dest_member -> src_member
    last_ref_member = None

    in_summary = False

    def finish_update_block():
        nonlocal in_pop_update, update_epoch, pending_updates, copied_from, last_ref_member
        if not in_pop_update:
            return
        # Apply pending updates to state for the next epoch (state is used when we hit the next epoch's TRAIN_MEMBER blocks).
        for dest, upd in pending_updates.items():
            ensure(dest)
            base = state[dest].copy()
            if dest in copied_from:
                src = copied_from[dest]
                ensure(src)
                # inherit from source when available
                if len(state[src]) > 0:
                    base = state[src].copy()
            base.update(upd)
            state[dest] = base
        # reset
        in_pop_update = False
        update_epoch = None
        pending_updates = {}
        copied_from = {}
        last_ref_member = None

    for ln in lines:
        m = EPOCH_HDR.search(ln)
        if m and "Summary" not in ln:
            # entering a new epoch: close any pending update blocks
            finish_update_block()
            in_summary = False
            current_member = None
            current_epoch = int(m.group("epoch"))
            epoch_total = int(m.group("epoch_total"))
            continue

        m = POP_UPDATE_HDR.search(ln)
        if m:
            finish_update_block()
            in_pop_update = True
            update_epoch = int(m.group("epoch"))
            pending_updates = {}
            copied_from = {}
            last_ref_member = None
            continue

        m = SUMMARY_HDR.search(ln)
        if m:
            in_summary = True
            summary_rows.append({"seed": seed, "epoch": int(m.group("epoch"))})
            continue

        if in_summary:
            srow = summary_rows[-1]
            m = SUMMARY_TIME.search(ln)
            if m:
                srow["time_s"] = float(m.group("time"))
                srow["avg_member_s"] = float(m.group("avg"))
                continue
            m = POP_MEAN_ACC.search(ln)
            if m:
                srow["pop_mean_acc_pct"] = float(m.group("acc"))
                continue
            m = BEST_MEMBER_ACC.search(ln)
            if m:
                srow["best_member_acc_pct"] = float(m.group("acc"))
                continue
            m = MEAN_BS.search(ln)
            if m:
                srow["mean_batch_size"] = int(m.group("bs"))
                continue
            m = MEAN_LR.search(ln)
            if m:
                srow["mean_lr"] = float(m.group("lr"))
                continue
            m = MEAN_WD.search(ln)
            if m:
                srow["mean_weight_decay"] = float(m.group("wd"))
                continue
            continue

        if in_pop_update:
            m = CHANGE_LINE.search(ln)
            if m:
                mem = int(m.group("member"))
                param = _norm_param_name(m.group("param"))
                old = float(m.group("old"))
                new = float(m.group("new"))
                changes.append({"seed": seed, "update_epoch": update_epoch, "member": mem, "param": param, "old": old, "new": new})
                if param not in ("lr", "batch_size"):
                    pending_updates.setdefault(mem, {})[param] = new
                last_ref_member = mem
                continue

            m = COPIED_LINE.search(ln)
            if m:
                dest = int(m.group("member"))
                src = int(m.group("src"))
                copied_from[dest] = src
                copies.append({"seed": seed, "update_epoch": update_epoch, "member": dest, "src_member": src})
                last_ref_member = dest
                continue

            m = POST_LINE.search(ln)
            if m:
                dest = last_ref_member
                if dest is None and len(copied_from) == 1:
                    dest = list(copied_from.keys())[0]
                if dest is not None:
                    wd = float(m.group("wd"))
                    dp = float(m.group("dp"))
                    warm = int(m.group("warm"))
                    pending_updates.setdefault(dest, {})
                    pending_updates[dest].update({"weight_decay": wd, "drop_path": dp, "warmup_epochs": warm})

                    post_rows.append({
                        "seed": seed,
                        "update_epoch": update_epoch,
                        "member": dest,
                        "lr": float(m.group("lr")),
                        "weight_decay": wd,
                        "drop_path": dp,
                        "warmup_epochs": warm,
                        "batch_size": int(m.group("bs")),
                    })

                    # Backfill source member's non-(LR, batch) hyperparams for previous epochs, as requested.
                    if dest in copied_from:
                        src = copied_from[dest]
                        anchors.append({"member": src, "epoch": update_epoch, "weight_decay": wd, "drop_path": dp, "warmup_epochs": warm})
                continue

            continue

        # Per-member training block
        m = TRAIN_MEMBER.search(ln)
        if m:
            current_member = int(m.group("member"))
            bs = int(m.group("bs"))
            ensure(current_member)
            rows.append({
                "seed": seed,
                "epoch": current_epoch,
                "epoch_total": epoch_total,
                "member": current_member,
                "train_batch_size": bs,
                "data_build_s": np.nan,
                "train_time_s": np.nan,
                "eval_time_s": np.nan,
                "loss": np.nan,
                "train_acc_pct": np.nan,
                "test_acc_pct": np.nan,
                # LR + batch ONLY from this block:
                "lr_sched_start": np.nan,
                "lr_sched_end": np.nan,
                "pbt_lr": np.nan,
                "pbt_batch_size": bs,
                # other hyperparams from population updates:
                "pbt_weight_decay": state[current_member].get("weight_decay", np.nan),
                "pbt_drop_path": state[current_member].get("drop_path", np.nan),
                "pbt_warmup_epochs": state[current_member].get("warmup_epochs", np.nan),
            })
            last_row_idx = len(rows) - 1
            continue

        if current_member is not None and last_row_idx is not None:
            m = BUILT.search(ln)
            if m:
                rows[last_row_idx]["data_build_s"] = float(m.group("t"))
                continue
            m = LR_CHANGED.search(ln)
            if m:
                start = float(m.group("start"))
                end = float(m.group("end"))
                rows[last_row_idx]["lr_sched_start"] = start
                rows[last_row_idx]["lr_sched_end"] = end
                # define per-epoch LR as end-of-epoch LR (consistent per your examples)
                rows[last_row_idx]["pbt_lr"] = end
                continue
            m = TRAIN_TIME.search(ln)
            if m:
                rows[last_row_idx]["train_time_s"] = float(m.group("t"))
                continue
            m = EVAL_TIME.search(ln)
            if m:
                rows[last_row_idx]["eval_time_s"] = float(m.group("t"))
                continue
            m = LOSS.search(ln)
            if m:
                rows[last_row_idx]["loss"] = float(m.group("loss"))
                continue
            m = TRAIN_ACC.search(ln)
            if m:
                rows[last_row_idx]["train_acc_pct"] = float(m.group("acc"))
                continue
            m = TEST_ACC.search(ln)
            if m:
                rows[last_row_idx]["test_acc_pct"] = float(m.group("acc"))
                continue

    # close any trailing update block
    finish_update_block()

    df_main = pd.DataFrame(rows)
    df_changes = pd.DataFrame(changes)
    df_copies = pd.DataFrame(copies)
    df_post = pd.DataFrame(post_rows)
    df_summary = pd.DataFrame(summary_rows)

    if len(df_main):
        df_main = df_main.sort_values(["member", "epoch"]).reset_index(drop=True)

        # Apply anchors (backfill sources for epochs <= update_epoch where missing)
        if len(anchors):
            df_anchor = pd.DataFrame(anchors)
            for mem, sub in df_anchor.groupby("member"):
                sub = sub.sort_values("epoch")
                a_epoch = int(sub.iloc[0]["epoch"])
                for col, src_col in [("pbt_weight_decay", "weight_decay"), ("pbt_drop_path", "drop_path"), ("pbt_warmup_epochs", "warmup_epochs")]:
                    val = float(sub.iloc[0][src_col])
                    mask = (df_main["member"] == mem) & (df_main["epoch"] <= a_epoch) & (df_main[col].isna())
                    df_main.loc[mask, col] = val

        # Fill LR forward (in case any epochs miss an LR line)
        for col in ["pbt_lr", "lr_sched_start", "lr_sched_end"]:
            df_main[col] = df_main.groupby("member")[col].ffill()

        # Batch size is taken from training blocks
        df_main["pbt_batch_size"] = df_main["train_batch_size"]

        # Non-(LR, batch) hyperparams: forward fill, then backfill leading NaNs per member
        for col in ["pbt_weight_decay", "pbt_drop_path", "pbt_warmup_epochs"]:
            df_main[col] = df_main.groupby("member")[col].ffill()
            df_main[col] = df_main.groupby("member")[col].bfill()

        df_main = df_main.sort_values(["epoch", "member"]).reset_index(drop=True)

    return {
        "seed": seed,
        "main": df_main,
        "changes": df_changes,
        "copies": df_copies,
        "post": df_post,
        "summary": df_summary,
    }

def parse_pbt_log(path: Path) -> dict:
    seed = parse_seed_from_name(path)
    text = path.read_text(encoding="utf-8", errors="ignore")
    return parse_pbt_text(text, seed)


## Run parser for all logs, concatenate, and export

Outputs:
- CSV: `results/pbt_bs_wd_parsed.csv`
- XLSX: `results/pbt_bs_wd_parsed.xlsx` with:
  - `epoch_member_metrics` (all seeds)
  - `epoch_summary`, `hyperparam_changes`, `copy_events`, `post_update_hparams` (all seeds, with seed column)
  - `leaderboard_top200` (combined)
  - `leaderboard_per_member` (combined)
  - `top200_<seed>` (per-seed top 200 snapshots)


In [3]:
# Parse each log
parsed = [parse_pbt_log(p) for p in INPUT_LOG_PATHS]
parsed = sorted(parsed, key=lambda d: d["seed"])
seed_values = [d["seed"] for d in parsed]

df_all = pd.concat([d["main"] for d in parsed], ignore_index=True) if parsed else pd.DataFrame()
df_summary_all = pd.concat([d["summary"] for d in parsed if len(d["summary"])], ignore_index=True)
df_changes_all = pd.concat([d["changes"] for d in parsed if len(d["changes"])], ignore_index=True)
df_copies_all = pd.concat([d["copies"] for d in parsed if len(d["copies"])], ignore_index=True)
df_post_all = pd.concat([d["post"] for d in parsed if len(d["post"])], ignore_index=True)

# Write CSV
df_all.to_csv(OUTPUT_CSV_PATH, index=False)

def leaderboard_top(df_in: pd.DataFrame, topn: int = 200) -> pd.DataFrame:
    snap = df_in.dropna(subset=["test_acc_pct"]).copy()
    snap = snap.sort_values(["test_acc_pct", "epoch"], ascending=[False, True]).reset_index(drop=True)
    snap.insert(0, "rank", snap.index + 1)
    return snap.head(topn)

def leaderboard_per_member(df_in: pd.DataFrame) -> pd.DataFrame:
    if not len(df_in):
        return pd.DataFrame()
    return (
        df_in.dropna(subset=["test_acc_pct"])
            .sort_values(["seed", "member", "test_acc_pct", "epoch"], ascending=[True, True, False, True])
            .groupby(["seed", "member"], as_index=False)
            .head(1)
            .reset_index(drop=True)
    )

lb_combined = leaderboard_top(df_all, topn=200)
lb_per_member = leaderboard_per_member(df_all)

# Write XLSX
with pd.ExcelWriter(OUTPUT_XLSX_PATH, engine="openpyxl") as writer:
    df_all.to_excel(writer, sheet_name="epoch_member_metrics", index=False)
    if len(df_summary_all): df_summary_all.to_excel(writer, sheet_name="epoch_summary", index=False)
    if len(df_changes_all): df_changes_all.to_excel(writer, sheet_name="hyperparam_changes", index=False)
    if len(df_copies_all): df_copies_all.to_excel(writer, sheet_name="copy_events", index=False)
    if len(df_post_all): df_post_all.to_excel(writer, sheet_name="post_update_hparams", index=False)

    lb_combined.to_excel(writer, sheet_name="leaderboard_top200", index=False)
    lb_per_member.to_excel(writer, sheet_name="leaderboard_per_member", index=False)

    for seed in seed_values:
        leaderboard_top(df_all[df_all["seed"] == seed], topn=200).to_excel(writer, sheet_name=f"top200_{seed}"[:31], index=False)

print("Seeds parsed:", seed_values)
print("Wrote CSV:", OUTPUT_CSV_PATH.resolve())
print("Wrote XLSX:", OUTPUT_XLSX_PATH.resolve())
df_all.head()


Seeds parsed: [38042, 217401]
Wrote CSV: /Users/etaashpatel/Documents/Final Project/Structured Outputs/PBT/pbt_bs_parsed.csv
Wrote XLSX: /Users/etaashpatel/Documents/Final Project/Structured Outputs/PBT/pbt_bs_parsed.xlsx


,seed,epoch,epoch_total,member,train_batch_size,data_build_s,train_time_s,eval_time_s,loss,train_acc_pct,test_acc_pct,lr_sched_start,lr_sched_end,pbt_lr,pbt_batch_size,pbt_weight_decay,pbt_drop_path,pbt_warmup_epochs
0,38042,1,70,0,64,1.649991,32.208708,2.335430,1.8976,28.81,34.96,4.840000e-08,0.000038,0.000038,64,0.038,0.057,5.0
1,38042,1,70,1,128,1.644381,15.774836,2.269814,1.8932,29.39,34.91,1.340000e-07,0.000052,0.000052,128,0.129,0.023,5.0
2,38042,1,70,2,256,1.623972,10.251626,2.268690,1.9062,28.87,36.16,4.750000e-07,0.000094,0.000094,256,0.027,0.094,5.0
3,38042,1,70,3,64,1.653538,32.429775,2.263580,1.8608,30.29,36.60,6.710000e-08,0.000053,0.000053,64,0.038,0.074,5.0
4,38042,1,70,4,128,1.668840,16.994755,2.271158,1.8833,29.75,34.49,1.820000e-07,0.000072,0.000072,128,0.027,0.094,5.0
